## Exploratory Data Analysis

In [1]:
import pandas as pd

In [2]:
beers = pd.read_csv('beers.csv')

In [3]:
beers.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [4]:
beers.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
beers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 7 columns):
abv           2348 non-null float64
ibu           1405 non-null float64
id            2410 non-null int64
name          2410 non-null object
style         2405 non-null object
brewery_id    2410 non-null int64
ounces        2410 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 131.9+ KB


In [6]:
beers['brewery_id'].nunique()

558

In [7]:
breweries = pd.read_csv('breweries.csv')

In [8]:
breweries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 4 columns):
Unnamed: 0    558 non-null int64
name          558 non-null object
city          558 non-null object
state         558 non-null object
dtypes: int64(1), object(3)
memory usage: 17.6+ KB


In [9]:
breweries.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [10]:
breweries = breweries.rename(columns={"Unnamed: 0": "brewery_id"})
breweries.head()

,brewery_id,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [11]:
breweries['brewery_id'].nunique()

558

In [12]:
df = pd.merge(breweries, beers, how="left", on="brewery_id")

In [13]:
df.head()

,brewery_id,name_x,city,state,abv,ibu,id,name_y,style,ounces
0,0,NorthGate Brewing,Minneapolis,MN,0.045,50.0,2692,Get Together,American IPA,16.0
1,0,NorthGate Brewing,Minneapolis,MN,0.049,26.0,2691,Maggie's Leap,Milk / Sweet Stout,16.0
2,0,NorthGate Brewing,Minneapolis,MN,0.048,19.0,2690,Wall's End,English Brown Ale,16.0
3,0,NorthGate Brewing,Minneapolis,MN,0.060,38.0,2689,Pumpion,Pumpkin Ale,16.0
4,0,NorthGate Brewing,Minneapolis,MN,0.060,25.0,2688,Stronghold,American Porter,16.0


In [14]:
df = df.rename(columns={"name_x": "brewery_name", "name_y":"beer_name"})

In [15]:
df.head()

,brewery_id,brewery_name,city,state,abv,ibu,id,beer_name,style,ounces
0,0,NorthGate Brewing,Minneapolis,MN,0.045,50.0,2692,Get Together,American IPA,16.0
1,0,NorthGate Brewing,Minneapolis,MN,0.049,26.0,2691,Maggie's Leap,Milk / Sweet Stout,16.0
2,0,NorthGate Brewing,Minneapolis,MN,0.048,19.0,2690,Wall's End,English Brown Ale,16.0
3,0,NorthGate Brewing,Minneapolis,MN,0.060,38.0,2689,Pumpion,Pumpkin Ale,16.0
4,0,NorthGate Brewing,Minneapolis,MN,0.060,25.0,2688,Stronghold,American Porter,16.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2410 entries, 0 to 2409
Data columns (total 10 columns):
brewery_id      2410 non-null int64
brewery_name    2410 non-null object
city            2410 non-null object
state           2410 non-null object
abv             2348 non-null float64
ibu             1405 non-null float64
id              2410 non-null int64
beer_name       2410 non-null object
style           2405 non-null object
ounces          2410 non-null float64
dtypes: float64(3), int64(2), object(5)
memory usage: 207.1+ KB


In [17]:
df.isna().sum()

brewery_id         0
brewery_name       0
city               0
state              0
abv               62
ibu             1005
id                 0
beer_name          0
style              5
ounces             0
dtype: int64

In [18]:
df['ibu'] = df['ibu'].fillna(0)

In [19]:
df.isna().sum()

brewery_id       0
brewery_name     0
city             0
state            0
abv             62
ibu              0
id               0
beer_name        0
style            5
ounces           0
dtype: int64

In [20]:
df.dropna(inplace=True)

In [21]:
df.isna().sum()

brewery_id      0
brewery_name    0
city            0
state           0
abv             0
ibu             0
id              0
beer_name       0
style           0
ounces          0
dtype: int64

In [22]:
df.shape

(2346, 10)

In [23]:
df.head()

,brewery_id,brewery_name,city,state,abv,ibu,id,beer_name,style,ounces
0,0,NorthGate Brewing,Minneapolis,MN,0.045,50.0,2692,Get Together,American IPA,16.0
1,0,NorthGate Brewing,Minneapolis,MN,0.049,26.0,2691,Maggie's Leap,Milk / Sweet Stout,16.0
2,0,NorthGate Brewing,Minneapolis,MN,0.048,19.0,2690,Wall's End,English Brown Ale,16.0
3,0,NorthGate Brewing,Minneapolis,MN,0.060,38.0,2689,Pumpion,Pumpkin Ale,16.0
4,0,NorthGate Brewing,Minneapolis,MN,0.060,25.0,2688,Stronghold,American Porter,16.0


In [25]:
df.describe()

,brewery_id,abv,ibu,id,ounces
count,2346.000000,2346.000000,2346.000000,2346.000000,2346.000000
mean,230.350384,0.059776,25.560102,1438.808184,13.593052
std,156.214608,0.013547,29.022453,748.618619,2.333117
min,0.000000,0.001000,0.000000,1.000000,8.400000
25%,93.000000,0.050000,0.000000,827.250000,12.000000
50%,205.000000,0.056000,18.000000,1462.500000,12.000000
75%,365.000000,0.067000,42.000000,2077.750000,16.000000
max,557.000000,0.128000,138.000000,2692.000000,32.000000
